## Spanish Distribution Grid

In [ ]:
# 12707 buses / nodes
# bus_geodata = pd.read_csv(grid_path / "Buscord_indexed.txt", sep=r"\s+", names=["Index", "X", "Y"], index_col=0)

# # 8087 loads
# # Grouped into 1138 power supplies (e.g. corresponding to a Residential Building)
# loads = pd.read_csv(grid_path / "Load_indexed.txt", sep=r"\s+", names=["Version", "Name", "Phases", "Bus", "Voltage", "Power", "PF", "Daily"])
# # Map each load to load profile
# load_shapes = pd.read_csv(grid_path / "Loadshape.txt", names=["Version", "Name", "No. of Points", "Min Interval", "File", "Use"], sep=r"\s+")

# # 135 Fuses (on lines)
# fuses = pd.read_csv(grid_path / "fuse_indexed.txt", names=["Version", "Name", "Bus1", "Bus2", "Phases", "LInecode", "Length", "Units"], sep=r"\s+")
# # 30 Transformers
# trafo = pd.read_csv(grid_path / "Transformer_ind.txt", sep=r"\s+",
#                     names=["Version", "Name", "Windings", "Buses", "Buses2", "Connection", "Connection2", "Voltages", "Voltages2", "Currents", "Currents2", "XHL", "sub"])
# reactors = pd.read_csv(grid_path / "neutral_ind.txt", sep=r"\s+")
# # 29 MV Lines (each has an energy meter)
# mv_lines = pd.read_csv(grid_path / "source2txline_ind.txt", sep=r"\s+")
# # 10138 LV Lines
# lv_lines = pd.read_csv(grid_path / "Line_indexed_check.txt", sep=r"\s+")
# lv_line_monitors = pd.read_csv(grid_path / "monitor_line.txt", sep=r"\s+")

In [ ]:
import pandas as pd
import pandapower as pp
from tqdm.notebook import trange, tqdm
from pathlib import Path
from opendssdirect import dss
grid_path = Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss"
print('OpenDSSDirect.py and engine versions:', dss.Version())
dss(f"Redirect '{str(grid_path / 'Master.dss')}'")

In [ ]:
import inspect

dss.Circuit.AllBusNames()
dss.Settings.VoltageBases()

dss.Transformers.AllNames()
dss.Reactors.AllNames()
dss.Loads.AllNames()
inspect.getmembers(dss.Circuit) # dss.Bus.NumNodes()

# Not Used: Storages, CNData, Capacitors, Fuses, GICSources, Generators,
#           Isource, PVsystems, TSData, Reclosers, WireData
# Loads, LoadShape
# Lines, LineCodes, LineGeometries, LineSpacings
# dss.Circuit.AllBusVolts(), dss.AllBusVMag
dss.Circuit.AllBusDistances()
dss.Circuit.NumBuses() # 10289 buses
dss.Circuit.TotalPower()
dss.Circuit.AllBusMagPu()
[name for name, _ in inspect.getmembers(dss) if not name.startswith("_")]

In [ ]:

net = pp.create_empty_network()

dss.Command("MakeBusList") # Force instantiation of all buses
dss.Command("CalcVoltageBases") # Auto Set Voltage Bases
for i, bus_name in tqdm(enumerate(dss.Circuit.AllBusNames()), total=dss.Circuit.NumBuses()):
    dss.Circuit.SetActiveBus(bus_name)
    kV = dss.Bus.kVBase()
    X, Y = dss.Bus.X(), dss.Bus.Y()
    # print(bus_name, kV, X, Y, dss.Bus.Nodes())
    pp.create_bus(net, kV, name=bus_name, geodata=(X,Y), type="b")
    if i == 100: break
    # break

In [ ]:
for i, line in enumerate(dss.Lines):
    if i < 1100:
        continue
    name = line.Name()
    idx = line.Idx()
    bus1_name, *bus1_terminals = line.Bus1().split(".")
    bus2_name, *bus2_terminals = line.Bus2().split(".")[0]
    length = line.Length()
    line.Units()
    C0, C1 = line.C0(), line.C1()
    is_switch = line.IsSwitch()
    R0, R1 = line.R0(), line.R1()
    Rg = line.Rg()
    X0, X1 = line.X0(), line.X1() # Per unit length
    Xg = line.Xg()
    n_phases = line.Phases()
    rho = line.Rho() # Resistivity
    normAmps, extAmps = line.NormAmps(), line.EmergAmps()
    print(line.Name(), line.Bus1(), line.Bus2(), line.Length(), C0, C1, R0, R1, Rg, X0, X1, Xg,
          line.NormAmps())
    # pp.create_line_from_parameters(net, from_bus, to_bus, length_km, r_ohm_per_km=R0, x_ohm_per_km=X0,
    #                                c_nf_per_km=C0, max_i_ka=normAmps/1000.0, name=name, index=idx, type=None,
    #                                geodata=None, in_service=True, df=1., parallel=n_phases, g_us_per_km=0.,
    #                                max_loading_percent=100.0,
    #                                r0_ohm_per_km=R0, x0_ohm_per_km=X0, c0_nf_per_km=C0)
    if i > 1150:
        break

In [71]:
for i, load in enumerate(dss.Loads):
    name = load.Name()
    size = load.kW()
    pf = load.PF()
    n_phases = load.Phases()
    load.Model
    print(name, size, load.kWh())
    # pp.create_load(net, bus, p_mw, q_mvar=0, name=name, )
    load.Bus
    break

load1 1.0 0.0
